In [1]:
from posthook import execute_posthook
execute_posthook()

In [1]:
!python main.py

^C


In [ ]:
from faang_stock_market_prices import get_faang_historical_prices
from database_handler import create_connection
from datetime import datetime
from lookups import CHROME_EXECUTOR

etl_date = datetime(2023,10,23)
db_session = create_connection()
get_usa_webscrapping_data(db_session,etl_datetime=etl_date,does_etl_exists=True,chrome_exec_path = CHROME_EXECUTOR.PATH)

In [6]:
from yahoofinancials import YahooFinancials
from database_handler import create_connection
from faang_stock_market_prices import get_latest_datetime_from_stock_price_table,convert_local_to_utc
from database_handler import execute_query,return_query,parse_date_columns
from pandas_data_handler import return_create_statement_from_df,return_insert_into_sql_statement_from_df
from datetime import datetime 

from lookups import DestinationDatabase
import pandas as pd
dst_schema = DestinationDatabase.SCHEMA_NAME.value
# etl_datetime = datetime(2023,10,23)

def create_staging_table(db_session,staging_df,schema_name,table_title):

    create_stmt = return_create_statement_from_df(dataframe= staging_df,schema_name = schema_name,table_name= table_title)
    execute_query(db_session=db_session, query= create_stmt)

def store_into_staging_table(db_session,staging_df,dst_schema,dst_table):

    if len(staging_df):
        insert_stmt = return_insert_into_sql_statement_from_df(staging_df,dst_schema, dst_table)
        execute_query(db_session=db_session, query= insert_stmt)
    


db_session = create_connection()
# latest_datetime = get_latest_datetime_from_stock_price_table(db_session,dst_schema)
latest_datetime = datetime(2007,1,1)
print("latest_datetime")
print(latest_datetime)
if not latest_datetime:
    latest_datetime = convert_local_to_utc(etl_datetime)


print("latest_datetime after processing",latest_datetime)
if latest_datetime.weekday() == 5:
    latest_datetime += timedelta(days=1)
# latest_date_utc = convert_local_to_utc(latest_datetime)

latest_date_str = latest_datetime.strftime('%Y-%m-%d')
print('latest_date after conversion')
print(latest_date_str)

end_datetime = convert_local_to_utc(datetime.today())
# end_datetime = convert_local_to_utc(datetime(2010,1,1))
end_date_str = end_datetime.strftime('%Y-%m-%d')

print('end_date_str after conversion')
print(end_date_str)

# tickers = ['META','AMZN','AAPL','NFLX','GOOGL']
tickers = ['META']
data = {}
for ticker in tickers:
    yahoo_financials = YahooFinancials(ticker)
    print("getting data for ",ticker)
    historical_data = yahoo_financials.get_historical_price_data(latest_date_str, end_date_str, "daily")
    print(historical_data)
    if historical_data[ticker]:
        data[ticker] = historical_data[ticker]['prices']
        print(data[ticker])
# print(date)
for ticker, prices in data.items():
    df = pd.DataFrame(prices)
    parse_date_columns(df)
    df = df.drop('date', axis=1).set_index('formatted_date')
    df['volume'] = df['volume'].astype(float)
    df.dropna(inplace=True)
    print(df)
    # source = 'yahoo_finance'
    # df_name = ticker + '_stock_price'
    # dst_table = f"stg_{source}_{df_name}"
    # create_staging_table(db_session = db_session,staging_df = df,schema_name =dst_schema,table_title = dst_table)
    # store_into_staging_table(db_session = db_session, staging_df = df, dst_schema = dst_schema ,dst_table = dst_table)
        

latest_datetime
2007-01-01 00:00:00
latest_datetime after processing 2007-01-01 00:00:00
latest_date after conversion
2007-01-01
end_date_str after conversion
2023-10-25
getting data for  META
{'META': {'eventsData': {}, 'firstTradeDate': {'formatted_date': '2012-05-18', 'date': 1337347800}, 'currency': 'USD', 'instrumentType': 'EQUITY', 'timeZone': {'gmtOffset': -14400}, 'prices': [{'date': 1337347800, 'high': 45.0, 'low': 38.0, 'open': 42.04999923706055, 'close': 38.22999954223633, 'volume': 573576400, 'adjclose': 38.22999954223633, 'formatted_date': '2012-05-18'}, {'date': 1337607000, 'high': 36.65999984741211, 'low': 33.0, 'open': 36.529998779296875, 'close': 34.029998779296875, 'volume': 168192700, 'adjclose': 34.029998779296875, 'formatted_date': '2012-05-21'}, {'date': 1337693400, 'high': 33.59000015258789, 'low': 30.940000534057617, 'open': 32.61000061035156, 'close': 31.0, 'volume': 101786600, 'adjclose': 31.0, 'formatted_date': '2012-05-22'}, {'date': 1337779800, 'high': 32.5

In [ ]:
from webscrape import get_finviz_news_webscrapping_data
from database_handler import create_connection
from datetime import datetime
df = get_finviz_news_webscrapping_data(create_connection(),datetime(2008,1,1),True)

In [ ]:
from sentiment_analysis import get_data_from_staging_table,analyze_sentiment
from lookups import FinvizWebScrape,DestinationDatabase,PoliticianSpeeches
from database_handler import create_connection
resource = PoliticianSpeeches  
df = get_data_from_staging_table(create_connection(), columns=resource.COLUMNS_NAME, source_name=resource.SOURCE,
                                         table_title=resource.TABLE_TITLE, destination_schema_name=DestinationDatabase.SCHEMA_NAME)
analyze_sentiment(df,resource.SOURCE,resource.TEXT_COLUMN_NAME)

In [ ]:
from gdp_arima_predict import get_forecast_gdp
from lookups import FredEconomicDataWebScrape,DestinationDatabase
from database_handler import create_connection
from hook import create_and_store_into_fact_table
df =get_forecast_gdp(create_connection())
# create_and_store_into_fact_table(create_connection(),df_title,DestinationDatabase.SCHEMA_NAME)

In [3]:
from sentiment_analysis import get_sentiment_analysis_results
from database_handler import create_connection
from lookups import FinvizWebScrape
df = get_sentiment_analysis_results(create_connection(),resources=[FinvizWebScrape])

c:\Users\user\OneDrive\Documents\SE_Factory\FSD\Final_Project\usa_recession_analysis\database_handler.py:64: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return_dataframe = pd.read_sql_query(con= db_session, sql= file_executor)


In [6]:
df[0][1]

,ticker,date,time,title,text,url,neg,neu,pos,compound
ticker_title,,,,,,,,,,
"META-Meta, CME Lead Top Stocks Breaking Out With This Bullish Trait",META,2023-10-07,08:00:00,"Meta, CME Lead Top Stocks Breaking Out With Th...","Meta, CME Lead Top Stocks Breaking Out With Th...",https://finance.yahoo.com/m/9e64308e-f93d-3c47...,0.0000,0.6190,0.3810,0.6249
META-Does ad-free Instagram mean the era of data harvesting is over?,META,2023-10-07,03:00:00,Does ad-free Instagram mean the era of data ha...,Does ad-free Instagram mean the era of data ha...,https://finance.yahoo.com/m/45d331ee-e14f-3de2...,-0.4134,0.0618,0.3105,-0.1427


In [1]:
from webscrape import get_usa_webscrapping_data,get_states_webscraping_data,get_finviz_news_webscrapping_data,get_politician_speeches
from database_handler import create_connection
import pandas as pd
from datetime import datetime
get_usa_webscrapping_data(create_connection(),datetime(2023, 10,14),True)

In [ ]:
import pandas as pd

# Creating a sample DataFrame
data = {'Name': ['Alice', 'Bob', 'Charlie'],
        'Age': [25, 30, 35],
        'City': ['New York', 'San Francisco', 'Los Angeles']}

df = pd.DataFrame(data)

# Creating an index using the 'Name' column
df.set_index('Name', inplace=True)

df


In [ ]:
df.index.name

# QUICK DEMO

In [ ]:
from webscrape import get_politician_speeches
from database_handler import create_connection
from datetime import datetime
from sentiment_analysis import store_sentiment_analysis_into_fact_table
from lookups import PoliticianSpeeches
df = get_politician_speeches(db_session=create_connection(),etl_datetime=datetime(2007,1,1))


In [ ]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import requests
from urllib.request import urlopen, Request
from urllib.parse import urlparse
from urllib.error import HTTPError
from lookups import FinvizWebScrape, ErrorHandling,DestinationDatabase,FredEconomicDataWebScrape,PoliticianSpeeches
from datetime import datetime,timedelta
from logging_handler import show_error_message
from pandas_data_handler import return_create_statement_from_df,return_insert_into_sql_statement_from_df,download_webscrape_csv_to_dataframe
from database_handler import execute_query,return_query,create_connection
from misc_handler import create_sql_table_index

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("C:\Program Files\Google\Chrome\Application\chrome.exe")
inner_driver = webdriver.Chrome(options=chrome_options)

url = 'https://millercenter.org/the-presidency/presidential-speeches/august-28-2008-acceptance-speech-democratic-national'
inner_driver.get(url)
response = requests.get(url)

max_wait_time = 10
schema_name = DestinationDatabase.SCHEMA_NAME.value
source_title = PoliticianSpeeches.SOURCE.value
table_title = PoliticianSpeeches.TABLE_TITLE.value
table_name = f"stg_{source_title}_{table_title}"

if response.status_code == 200:
    speech_title = 'August 28, 2008: Acceptance Speech at the Democratic National Convention'
    df = pd.DataFrame()
    speech_datetime = pd.to_datetime(speech_title.split(':')[0])
    df['date'] = [speech_datetime]
    df['speech_title'] = [speech_title.split(':')[1]]
    # WebDriverWait(inner_driver, max_wait_time).until(
    #     EC.presence_of_element_located((By.CLASS_NAME, "transcript-btn-inner"))
    # )

    # transcript_button = WebDriverWait(inner_driver, max_wait_time).until(
    #     EC.element_to_be_clickable((By.CLASS_NAME, "transcript-btn-inner"))
    # )
    # time.sleep(2)

    # transcript_button.click()

    df['speaker_name'] = ['Barack Obama']

    transcript_text = WebDriverWait(inner_driver, max_wait_time).until(
        EC.presence_of_element_located((By.CLASS_NAME, "view-transcript"))
    )
    
    paragraphs = transcript_text.find_elements(By.TAG_NAME, 'p')
    text = ''

    for paragraph in paragraphs:
        text+=paragraph.text + '\n'
    
    df['speech'] = [text]
    insert_stmt = return_insert_into_sql_statement_from_df(df,schema_name,table_name,is_upsert=True)
    execute_query(create_connection(),insert_stmt)
else:
    error_string_prefix = ErrorHandling.WEBSCRAPE_PAGE_FAILED.value
    error_string_suffix = f"Unable to web scrape {inner_url}, HTTP status code: " +response.getcode()
    show_error_message(error_string_prefix,error_string_suffix)

In [ ]:
from sentiment_analysis import preprocess_text
from nltk.sentiment.vader import SentimentIntensityAnalyzer

vader = SentimentIntensityAnalyzer()

# vader.polarity_scores(preprocess_text(df['speech'][0]))
scores = df['speech'].apply(preprocess_text).apply(
        vader.polarity_scores).tolist()

In [ ]:
scores

In [ ]:
import pandas as pd
scores_df = pd.DataFrame(scores)

In [ ]:
scores_df
# scores_df.index.name = "index"

In [ ]:
# df[['neg','neu','pos','compound']] = scores_df[['neg','neu','pos','compound']]

df['neg'] = scores_df['neg']

In [ ]:
scores_df['date'] = df.index

In [ ]:
scores_df

In [ ]:
df_join = df.join(scores_df)

In [ ]:
df_join = pd.merge(df, scores_df, on='date')

In [ ]:
df_join

In [ ]:
import pandas as pd

# Sample DataFrames
data1 = {
    ('A', 'one'): [1, 2, 3],
    ('A', 'two'): [4, 5, 6],
    ('B', 'one'): [7, 8, 9],
    ('B', 'two'): [10, 11, 12],
}
df1 = pd.DataFrame(data1, index=['X', 'Y', 'Z'])

data2 = {
    'C': [13, 14, 15],
    'D': [16, 17, 18],
}
df2 = pd.DataFrame(data2, index=['X', 'Y', 'Z'])

# Join on the index (assuming the index is common between the two DataFrames)
result = df1.join(df2)

# If the columns you want to join on are not the index, you can use the 'on' parameter
# For example, if the common column is 'key_column':
# result = df1.join(df2.set_index('key_column'), on=('A', 'one'))

df2

In [ ]:
from yahoofinancials import YahooFinancials
import pandas as pd
from datetime import datetime, date, time,timedelta
import pytz

# tickers = ['META','AMZN','AAPL','NFLX','GOOGL']
tickers = ['AMZN']

start_date = date(2023,10,11)
start_date = start_date.strftime('%Y-%m-%d')
print("start_date")
print(start_date)
end_date = datetime.today() + timedelta(days=1)
end_date = end_date.strftime('%Y-%m-%d')
print("end_date")
print(end_date)
data = {}
for ticker in tickers:
    yahoo_financials = YahooFinancials(ticker)
    historical_data = yahoo_financials.get_historical_price_data(start_date, end_date, "daily")
    if historical_data[ticker]:
        data[ticker] = historical_data[ticker]['prices']
dfs = []
titles = []
for ticker, prices in data.items():
    df = pd.DataFrame(prices)
    # parse_date_columns(df)
    df = df.drop('date', axis=1).set_index('formatted_date')
    df['volume'] = df['volume'].astype(float)
    # title = f'{get_coin_name(ticker[:-4])}_historical_price'
    # titles.append(title)
    dfs.append(df)
        

In [ ]:
dfs

In [ ]:
from database_handler import create_connection
from webscrape import get_last_news_date
db_session = create_connection()

get_last_news_date(db_session)

In [ ]:
start_date

In [ ]:
start_date = datetime.strptime('2023-01-01', '%Y-%m-%d').replace(tzinfo=pytz.UTC)
start_date

In [ ]:
from webscrape import get_states_webscrape_data_from_fred_economic
from datetime import datetime
db_session = create_connection()
get_states_webscrape_data_from_fred_economic(db_session,datetime(2007,1,1),False)

In [ ]:
end_date = datetime.today()
print(end_date)
local_tz = pytz.timezone('Asia/Beirut')
utc_datetime = local_tz.localize(end_date).astimezone(pytz.UTC)
print(utc_datetime)
utc_date = utc_datetime.date()
utc_date

In [ ]:
datetime.today().date().strftime('%Y-%m-%d')

In [ ]:
from database_handler import return_query,create_connection
db_session = create_connection()
query = """SELECT EXISTS (
   SELECT 1
   FROM information_schema.tables
   WHERE table_schema = 'dw_reporting'
     AND table_name = 'dim_faang_stock_price'
);"""
return_query(db_session = db_session,query =  query)[0][0]


In [ ]:
latest_date_query = f"""
SELECT
    MAX(formatted_date)
FROM dw_reporting.dim_faang_stock_price
"""
latest_date = return_query(db_session = db_session,query = latest_date_query)[0][0]
latest_date

In [ ]:
from datetime import timedelta
latest_date+ timedelta(days=1)

In [ ]:
news_df = scrape_website(etl_date= datetime(2023, 10, 6))

In [9]:
import pandas as pd
from datetime import datetime

# Function to generate quarterly dates within a given range
def generate_quarterly_dates(start_date, end_date):
    return pd.date_range(start=start_date, end=end_date, freq='Q')

# Create a data frame to hold the president names and dates
president_dates = pd.DataFrame({
    'President': ["George W. Bush", "Barack Obama", "Donald Trump", "Joe Biden"],
    'Start_Date': pd.to_datetime(["2001-01-20", "2009-01-20", "2017-01-20", "2021-01-20"]),
    'End_Date': pd.to_datetime(["2009-01-20", "2017-01-20", "2021-01-20", datetime.today()])
})

# Create an empty data frame to store the results
result_df = pd.DataFrame()

# Iterate through each president's term
for i in range(len(president_dates)):
    president_name = president_dates['President'][i]
    start_date = president_dates['Start_Date'][i]
    end_date = president_dates['End_Date'][i]

    # Generate quarterly dates within the president's term
    quarterly_dates = generate_quarterly_dates(start_date, end_date)

    # Create a data frame for the current president
    president_df = pd.DataFrame({
        'President': [president_name] * len(quarterly_dates),
        'Date': quarterly_dates
    })

    # Append the data for the current president to the result data frame
    result_df = pd.concat([result_df, president_df], ignore_index=True)

# result_df['Date'] = pd.to_datetime(result_df['Date'])
# Output the result data frame
result_df.dtypes

President            object
Date         datetime64[ns]
dtype: object

In [11]:
import pandas as pd
from datetime import datetime

# Function to generate quarterly dates within a given range
def generate_quarterly_dates(start_date, end_date):
    return pd.date_range(start=start_date, end=end_date, freq='Q')

# Create a data frame to hold the president names and dates
president_dates = pd.DataFrame({
    'President': ["George W. Bush", "Barack Obama", "Donald Trump", "Joe Biden"],
    'Start_Date': pd.to_datetime(["2001-01-20", "2009-01-20", "2017-01-20", "2021-01-20"]),
    'End_Date': pd.to_datetime(["2009-01-20", "2017-01-20", "2021-01-20", datetime.today()])
})

# Create an empty data frame to store the results
result_df = pd.DataFrame()

# Iterate through each president's term
for i in range(len(president_dates)):
    president_name = president_dates['President'][i]
    start_date = president_dates['Start_Date'][i]
    end_date = president_dates['End_Date'][i]

    # Generate quarterly dates within the president's term
    quarterly_dates = generate_quarterly_dates(start_date, end_date)

    # Create a data frame for the current president
    president_df = pd.DataFrame({
        'President': [president_name] * len(quarterly_dates),
        'Date': quarterly_dates
    })

    # Append the data for the current president to the result data frame
    result_df = pd.concat([result_df, president_df], ignore_index=True)

# Convert the 'Date' column to datetime format
result_df['Date'] = pd.to_datetime(result_df['Date'])

result_df

President            object
Date         datetime64[ns]
dtype: object